# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 4 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.

### Preamble

In [1]:
import numpy as np
import qutip as qt

### Exercise 4.1 

**Since the referee is purely classical, we'll model them as using classical random number generators.
This leaves open the possibility, though, that you and Eve could cheat by guessing the referee's questions.
A possible improvement might be to use the QRNGs from Chapter 2.
Modify the code sample in [`chsh.py`](./chsh.py) so that the referee can ask questions of you and Eve by measuring a qubit that starts off in the $\left|+\right\rangle$ state.**

The key to this exercise is replacing `random_bit`:

```python
def random_bit() -> int:
    return random.randint(0, 1)
```

You can use what you learned in Chapter 2 to replace this by a QRNG:

```python
def random_bit(device: QuantumDevice) -> int:
    with device.using_qubit() as q:
        q.h()
        return int(q.measure())
```

The referee will then need to allocate a new quantum device to use in generating their random bits:

```python
def referee(strategy: Callable[[], Strategy]) -> bool:
    device = Simulator(capacity=1)
    you, eve = strategy()
    your_input, eve_input = random_bit(device), random_bit(device)
    # ...
```

----
### Exercise 4.2

**How would you prepare a $\left|+0\right\rangle$ state?
First, what vector would you use to represent the two-qubit state $\left|+0\right\rangle = \left|+\right\rangle \otimes \left|0\right\rangle$?
You have an initial two qubit register in the $\left|00\right\rangle$ state.
What operation should you apply to get the state you want?**

*HINT*: try (𝐻 ⊗ 𝟙) if you are stuck!

For this exercise, you can start by writing out the vector representations of $|+\rangle$ and $|0\rangle$ as QuTiP `Qobj` instances:

In [2]:
ket0 = qt.Qobj([
    [1],
    [0]
])
ket_plus = qt.Qobj([
    [1],
    [1]
]) / np.sqrt(2)

You can then find the two-qubit state $|+0\rangle = |{+}\rangle \otimes |0\rangle$ using `qt.tensor`:

In [3]:
qt.tensor(ket_plus, ket0)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]

Using the hint above, you can verify that this is what you get if you apply the Hadamard operation (represented by the unitary matrix returned by `qt.qip.operations.hadamard_transform`) to the first qubit of a two-qubit register. Note that as mentioned in the hint, you'll need to act on the state of the second qubit with identity matrix to represent doing nothing. The identity matrix can be obtained using the QuTiP function `qt.qeye(2)`.

In [4]:
U = qt.tensor(qt.qip.operations.hadamard_transform(), qt.qeye(2))

You can then verify that applying the Hadamard operation to the first qubit does what you expect to the state of the two-qubit register.

In [5]:
U * qt.tensor(ket0, ket0)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]

Since this agrees with the output of `qt.tensor(ket_plus, ket0)`, you've confirmed that you can use the Hadamard operation on the first qubit in a two-qubit register to prepare $|+0\rangle = |+\rangle \otimes |0\rangle$

----
### Exercise 4.3

**How would you create a `Qobj` to represent the |1⟩ state? How about the |+⟩ or $\left|-\right\rangle$ state? If you need to check back to _Simulating qubits in code_ section of Chapter 2 for what vectors represent those states.**

In [6]:
ket1 = qt.Qobj([
    [0],
    [1]
])
ket1

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]

In [7]:
ket_plus = qt.Qobj([
    [1],
    [1]
]) / np.sqrt(2)
ket_plus

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

In [8]:
ket_minus = qt.Qobj([
    [1],
    [-1]
]) / np.sqrt(2)
ket_minus

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.70710678]
 [-0.70710678]]

----
### Exercise 4.4

**How could you use the `qt.basis` function to create a two qubit register in the |10⟩ state?
How could you create the $\left|001\right\rangle$ state? 
Remember that the second argument to `qt.basis` is an index to the computational basis states we saw earlier.**

In [9]:
ket001 = qt.tensor([qt.basis(2, label) for label in (0, 0, 1)])
ket001

Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

----
### Exercise 4.5

**In the example where our two qubits start off in the $\left|++\right\rangle$ state, suppose we measured the second qubit instead.
Check that no matter what result we get, nothing happens to the state of the first qubit.**

It's easiest to consider this in two cases, but first let's start by writing out $|++\rangle$ as a vector.

In [10]:
qt.tensor(ket_plus, ket_plus)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.5]
 [0.5]
 [0.5]
 [0.5]]

This vector has a non-zero amplitude on all four computational basis states $|00\rangle$, $|01\rangle$, $|10\rangle$, and $|11\rangle$.
In the case that measuring the second qubit gives us a "0" result, we need to filter out the amplitudes that are inconsistent with that result (in particular, the amplitudes of $|01\rangle$ and $|11\rangle$, represented by the second and fourth rows, respectively).

In [11]:
zero_case = qt.Qobj([
    [0.5],
    [0.0],
    [0.5],
    [0.0]
]).unit()

Above, we used the `unit` method to renormalize (that is, to make the length of the state vector equal to 1) so that the measurement probabilities still sum to one.

In [12]:
zero_case

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]

We note that this is equivalent to $|+0\rangle$:

In [13]:
qt.tensor(ket_plus, ket0)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]

Similarly, in the "1" case, we need to filter out the first and third rows:

In [14]:
one_case = qt.Qobj([
    [0.0],
    [0.5],
    [0.0],
    [0.5]
]).unit()
one_case

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.        ]
 [0.70710678]]

In [15]:
qt.tensor(ket_plus, ket1)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.        ]
 [0.70710678]]

In both cases, the state of the first qubit after measuring was still $|+\rangle$, confirming that measuring the second qubit had no effect on the state of the first qubit in this case.
Note that this may not have been the case if the two qubits started off in an *entangled state*.